<a href="https://www.kaggle.com/code/serhanakmak/notebook1c8a2985a5?scriptVersionId=121536684" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from bs4 import BeautifulSoup
import requests

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mpst-movie-plot-synopses-with-tags/partition.json
/kaggle/input/mpst-movie-plot-synopses-with-tags/mpst_full_data.csv
/kaggle/input/imdb-dataset-of-top-1000-movies-and-tv-shows/imdb_top_1000.csv


In [2]:

user = input("User name:\t")
url = "https://letterboxd.com/"+ user +"/films"

userDf = pd.DataFrame(columns= ["title", "ranking"])

def get_movies (url, userDf):
    response = requests.get(url)
    if response.status_code == 200:

        # Get the HTML content of the website
        html_content = response.content

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')
        check = userDf.shape
        # Print the title of the website
        for el in soup.findAll("li",class_="poster-container"):
            try:
                name =  el.find("img").get("alt")
                res  = el.find ("p", class_ ="poster-viewingdata").find("span")
               
                res = len(res.text)-0.5 if '½' in res.text else  len(res.text)
            except:
                continue
            userDf = userDf.append({"ranking": res, "title":name }, ignore_index=True)
                
        return userDf, check != userDf.shape

    else:
        # If the request was unsuccessful, print an error message
        print('Failed to retrieve website')
        return userDf, False
a = 1
while True:
    userDf, flag = get_movies(url,userDf)
    if (a==1):
        url+="/page/2"
    a += 1
    if (flag == False):
        break
    else:
        url = url[:-(len(str(a-1)))]+str(a)
userDf

User name:	 ahmetfirat


,title,ranking
0,The Menu,3.5
1,Everything Everywhere All at Once,4.5
2,The Batman,4.0
3,Spider-Man: No Way Home,3.5
4,Don't Look Up,3.5
...,...,...
180,It's a Wonderful Life,4.0
181,Citizen Kane,2.0
182,The Great Dictator,3.0
183,Modern Times,3.5


In [3]:
imdb = pd.read_csv("/kaggle/input/imdb-dataset-of-top-1000-movies-and-tv-shows/imdb_top_1000.csv")

search_list = list(userDf["title"])
def organizer(lst):
    tmp = imdb[imdb['Series_Title'].isin(lst)][[col for col in imdb if col not in ["Poster_Link", "Overview","Director", "Star1","Star2","Star3","Star4", "Certificate","Gross"]]]
    tmp["Runtime"] = tmp["Runtime"].str[:-4]
    tmp.rename(columns={"Series_Title": "title"}, inplace=True)
    return tmp
tmp = organizer(search_list)
imdb_final = pd.merge(tmp, userDf, on='title', how='outer')
imdb_final  =imdb_final.dropna()
imdb_final.head()

,title,Released_Year,Runtime,Genre,IMDB_Rating,Meta_score,No_of_Votes,ranking
0,The Shawshank Redemption,1994,142,Drama,9.3,80.0,2343110.0,3.5
1,The Dark Knight,2008,152,"Action, Crime, Drama",9.0,84.0,2303232.0,4.5
2,12 Angry Men,1957,96,"Crime, Drama",9.0,96.0,689845.0,4.5
3,The Lord of the Rings: The Return of the King,2003,201,"Action, Adventure, Drama",8.9,94.0,1642758.0,4.5
4,Pulp Fiction,1994,154,"Crime, Drama",8.9,94.0,1826188.0,4.5


In [4]:
#while the large number of resulting columns may be a concern, it is still logical 
#to use one-hot encoding to handle categorical data, as it can often lead to better 
#performance than other encoding methods.
categories = imdb_final['Genre'].str.split(', ')
one_hot = pd.get_dummies(categories.apply(pd.Series).stack()).groupby(level=0).sum()
encoded = pd.concat([imdb_final, one_hot], axis=1)
encoded = encoded.astype({"Runtime": int,"Released_Year": int })
encoded.head()

,title,Released_Year,Runtime,Genre,IMDB_Rating,Meta_score,No_of_Votes,ranking,Action,Adventure,...,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,The Shawshank Redemption,1994,142,Drama,9.3,80.0,2343110.0,3.5,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Dark Knight,2008,152,"Action, Crime, Drama",9.0,84.0,2303232.0,4.5,1,0,...,0,0,0,0,0,0,0,0,0,0
2,12 Angry Men,1957,96,"Crime, Drama",9.0,96.0,689845.0,4.5,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Lord of the Rings: The Return of the King,2003,201,"Action, Adventure, Drama",8.9,94.0,1642758.0,4.5,1,1,...,0,0,0,0,0,0,0,0,0,0
4,Pulp Fiction,1994,154,"Crime, Drama",8.9,94.0,1826188.0,4.5,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
from sklearn.model_selection import train_test_split

In [6]:
cols_to_use = [col for col in encoded.columns if col not in ["title", "ranking", "Genre"] ]
X = encoded[cols_to_use]
y = encoded["ranking"]
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [7]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

In [8]:
param_grid = {
    'n_estimators': [50, 100, 500],
    'learning_rate': [ 0.5, 0.1, 0.01]
}
my_model = XGBRegressor( n_jobs=4)
my_model.fit(X_train, y_train, 
     early_stopping_rounds=5, 
     eval_set=[(X_valid, y_valid)], 
     verbose=False)
grid_search = GridSearchCV(estimator=my_model, param_grid=param_grid, cv=5)
grid_search.fit(X, y)
print("Best hyperparameters: ", grid_search.best_params_)
print("Mean absolute error: ", -grid_search.best_score_)

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


Best hyperparameters:  {'learning_rate': 0.5, 'n_estimators': 50}
Mean absolute error:  0.8292250919708659


In [10]:
while True:
    movie =  input("The movie name you want to know the possible ranking of:\t") 
    if movie =="q":
        break
    try:
        db_search = organizer([movie])
        category = db_search['Genre'].str.split(', ')
        one_hot2 = pd.get_dummies(category.apply(pd.Series).stack()).groupby(level=0).sum()
        default_cols =  [col for col in one_hot.columns.tolist() if col not in one_hot2.columns.tolist()]
        one_hot2 = one_hot2.assign(**{col: 0 for col in default_cols})
        param = pd.concat([db_search, one_hot2], axis=1)[cols_to_use].astype({"Runtime": int,"Released_Year": int })
        predicted_ranking = grid_search.predict(param)
        print(f"You would rate {movie} {float(predicted_ranking):.2f}/5.00" )
    except Exception as e: 
        print(e)
        print("Movie search failed.")
    

The movie name you want to know the possible ranking of:	 The Godfather


You would rate The Godfather 2.85/5.00


The movie name you want to know the possible ranking of:	 Scarface


You would rate Scarface 4.22/5.00


The movie name you want to know the possible ranking of:	 Interstellar


...

The movie name you want to know the possible ranking of:	 Casablanca


You would rate Casablanca 2.55/5.00


The movie name you want to know the possible ranking of:	 Inception


You would rate Inception 4.00/5.00


The movie name you want to know the possible ranking of:	 q
